In [ ]:
#### Second attempt at finding an inflationary potential using a genetic algorithm

In [74]:
import numpy as np
import random
import matplotlib.pyplot as plotter

In [75]:
PHI_POINTS = 100
PHI_RANGE = (-5, 5)

def compute_potential(phi, coefficients):
    return sum(a * phi ** n for n, a in enumerate(coefficients))

def compute_derivatives(phi, coefficients):
    V_prime = sum(n * a * phi ** (n - 1) for n, a in enumerate(coefficients) if n > 0)
    V_double_prime = sum(n * (n - 1) * a * phi ** (n - 2) for n, a in enumerate(coefficients) if n > 1)
    return V_prime, V_double_prime

def compute_slowroll_params(phi, coefficients):
    V = compute_potential(phi, coefficients)
    Vp, Vpp = compute_derivatives(phi, coefficients)

    epsilon = 0.5 * (Vp / V)**2
    eta = Vpp / V

    if not np.isfinite(epsilon) or not np.isfinite(eta):
        return np.nan, np.nan

    return epsilon, eta

# def compute_observables(coefficients):
#     phi_vals = np.linspace(*PHI_RANGE, PHI_POINTS)
#     eps_vals = []
#     integrand = []
#     for phi in phi_vals:
#         try:
#             epsilon, eta = compute_slowroll_params(phi, coefficients)
#             if epsilon < 1e-10 or np.isnan(epsilon) or np.isnan(eta):  # avoid div/0
#                 continue
#             eps_vals.append(epsilon)
#             integrand.append(1 / np.sqrt(2 * epsilon))
#         except:
#             continue
#
#     if len(integrand) < 2:
#         return 0, 0, 1  # fail-safe fallback
#
#     Ne = np.trapezoid(integrand, phi_vals[:len(integrand)])
#     epsilon = np.mean(eps_vals)
#     ns = 1 - 6 * epsilon
#     r = 16 * epsilon
#     return Ne, ns, r

def compute_observables(coefficients):
    phi_vals = np.linspace(*PHI_RANGE, PHI_POINTS)
    eps_vals = []
    eta_vals = []
    integrand = []

    for phi in phi_vals:
        epsilon, eta = compute_slowroll_params(phi, coefficients)
        if epsilon < 1e-10 or np.isnan(epsilon) or np.isnan(eta):
            continue
        eps_vals.append(epsilon)
        eta_vals.append(eta)
        integrand.append(1 / np.sqrt(2 * epsilon))

    if len(integrand) < 2:
        return 0, 0, 1

    Ne = np.trapezoid(integrand, phi_vals[:len(integrand)])
    epsilon = np.mean(eps_vals)
    eta = np.mean(eta_vals)

    ns = 1 - 6 * epsilon + 2 * eta
    r = 16 * epsilon
    return Ne, ns, r

In [76]:
def fitness(coefficients):
    Ne, ns, r = compute_observables(coefficients)

    f = np.exp(-abs(Ne - 60)/10) + np.exp(-abs(ns - 0.965) * 100) + np.exp(-max(0, r - 0.07) * 200)
    return f

In [77]:
bound_a = 1.0
def generate_population(size, degree):
    return [np.random.uniform(-bound_a, bound_a, size=degree+1) for _ in range(size)]

In [78]:
# Most hyperparameters do not converge ns and r properly. Needs more work

POPULATION = 200
MUTATION_RATE = 0.2
POLYNOMIAL_DEGREE = 6
NUMBER_OF_GENERATIONS = 500
GENERATION_FILTER = 10

In [ ]:
|def crossover(p1, p2):
    alpha = np.random.rand()
    return alpha * p1 + (1 - alpha) * p2

def mutate(coefficients, rate=MUTATION_RATE):
    new = coefficients.copy()
    for i in range(len(new)):
        if np.random.rand() < rate:
            new[i] += np.random.normal(0, 0.2)
    return new

In [ ]:
population = generate_population(POPULATION, POLYNOMIAL_DEGREE)

for gen in range(NUMBER_OF_GENERATIONS):
    scored = [(fitness(ind), ind) for ind in population]
    scored.sort(reverse=True, key=lambda x: x[0])
    best = [ind for _, ind in scored[:GENERATION_FILTER]]


    children = best.copy()
    while len(children) < POPULATION:
        parents = random.sample(best, 2)
        child = crossover(parents[0], parents[1])
        children.append(mutate(child))
    population = children

    if gen % 5 == 0:
        Ne, ns, r = compute_observables(scored[0][1])
        print(f"Generation {gen+1}: Best fitness = {scored[0][0]:.4f}")
        print(f"Top candidate @ Gen {gen}: Ne={Ne:.2f}, ns={ns:.3f}, r={r:.4f}")


In [ ]:
best_fitness, best_coefficients = scored[0]
phi = np.linspace(*PHI_RANGE, PHI_POINTS)
V = compute_potential(phi, best_coefficients)

plotter.plot(phi, V)
plotter.title("Best Inflationary Potential Found")
plotter.xlabel("ϕ")
plotter.ylabel("V(ϕ)")
plotter.grid(True)
plotter.show()

Ne, ns, r = compute_observables(best_coefficients)
print("\nBest Candidate Observables:")
print(f"  Number of e-folds: {Ne:.2f}")
print(f"  Spectral index (ns): {ns:.4f}")
print(f"  Tensor-to-scalar ratio (r): {r:.4f}")
